In [38]:
# nl2sql_local_llama.py
# NL -> SQL pipeline (rule-based first) with local Llama 3.2 fallback (intent + entities)
# Uses safe SQL templates for your schema.
# Reference uploaded PDF: /mnt/data/3rd sem Lab Batch List.pdf

import re
import json
import os
import requests
import mysql.connector
from datetime import datetime, timedelta
from typing import Dict, Any, Tuple, Optional
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),      # "ollama"
    base_url=os.getenv("OPENAI_BASE_URL")     # "http://localhost:11434/v1"
)

# -------------------------
# CONFIG - change to your values
# -------------------------
MYSQL_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "naraayana",   
    "database": "ml_db_collection"
}

# Local Llama 3.2 server config (DEFAULTS)
# Examples of local servers:
# - Ollama default API: http://localhost:11434 (endpoint /api/chat)
# - OpenAI-compatible local inference: http://localhost:8080/v1/chat/completions
# ------------------------------
# LOCAL LLAMA 3.2 (via Ollama)
# OpenAI compatible endpoint
# ------------------------------

from openai import OpenAI
import os

# Uses .env
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),      # "ollama"
    base_url=os.getenv("OPENAI_BASE_URL")     # "http://localhost:11434/v1"
)

def llama_local_chat(prompt):
    """
    Calls local Llama 3.2 using OpenAI-compatible API in Ollama.
    Returns text or None if failure.
    """
    try:
        resp = client.chat.completions.create(
            model="llama3.2",  # IMPORTANT: must match your ollama model name
            messages=[
                {"role": "system", "content": "You are a classifier and entity extractor."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        return resp.choices[0].message.content.strip()

    except Exception as e:
        print("⚠ Local Llama call failed:", e)
        return None


In [39]:
def db_connect():
    return mysql.connector.connect(**MYSQL_CONFIG)

def run_sql_fetchall(query: str, params: dict = None):
    conn = db_connect()
    cur = conn.cursor(dictionary=True)
    cur.execute(query, params or {})
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return rows

def run_sql_execute(query: str, params: dict = None):
    conn = db_connect()
    cur = conn.cursor()
    cur.execute(query, params or {})
    conn.commit()
    cur.close()
    conn.close()

# -------------------------
# Whitelisted SQL templates
# (exactly match your schema)
# -------------------------
SQL_TEMPLATES = {
    "student_exam_room": """
        SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, e.exam_date, e.exam_name
        FROM student_allocation sa
        JOIN exam e ON sa.exam_id = e.exam_id
        WHERE sa.usn = %(usn)s
        {exam_name_filter}
    """,
    "student_exam_bench": """
        SELECT sa.room_id, sa.bench_number, sa.bench_place
        FROM student_allocation sa
        JOIN exam e ON sa.exam_id = e.exam_id
        WHERE sa.usn = %(usn)s
        {exam_name_filter}
    """,
    "student_full_schedule": """
        SELECT e.exam_name, e.exam_date, e.slot, sa.room_id, sa.bench_number, sa.bench_place
        FROM student_allocation sa
        JOIN exam e ON sa.exam_id = e.exam_id
        WHERE sa.usn = %(usn)s
        ORDER BY e.exam_date, e.slot
    """,
    "faculty_room_duty": """
        SELECT ra.room_id, rd.capacity, e.exam_name, e.exam_date, e.slot, ra.no_students
        FROM room_allocation ra
        JOIN exam e ON ra.exam_id = e.exam_id
        JOIN room_details rd ON rd.room_id = ra.room_id
        WHERE ra.fac_id = %(fac_id)s
        {date_slot_filter}
    """,
    "faculty_student_list": """
        SELECT sa.usn, s.s_name, sa.bench_number, sa.bench_place
        FROM student_allocation sa
        JOIN student s ON sa.usn = s.usn
        WHERE sa.fac_id = %(fac_id)s
        AND sa.exam_id = %(exam_id)s
        ORDER BY sa.bench_number, sa.bench_place
    """,
    "faculty_full_duties": """
        SELECT e.exam_name, e.exam_date, e.slot, ra.room_id, ra.no_students
        FROM room_allocation ra
        JOIN exam e ON ra.exam_id = e.exam_id
        WHERE ra.fac_id = %(fac_id)s
        ORDER BY e.exam_date, e.slot
    """,
    "unused_rooms": """
        SELECT rd.room_id, rd.capacity
        FROM room_details rd
        WHERE rd.room_id NOT IN (
            SELECT room_id FROM room_allocation ra
            JOIN exam e ON ra.exam_id = e.exam_id
            WHERE e.exam_date = %(exam_date)s AND e.slot = %(slot)s
        )
    """,
    "unallocated_students": """
        SELECT (SELECT COUNT(*) FROM student) - 
               (SELECT COUNT(*) FROM student_allocation WHERE exam_id = %(exam_id)s) AS not_allocated_students
    """,
    "room_remaining_capacity": """
        SELECT rd.room_id, rd.capacity, ra.no_students, (rd.capacity - ra.no_students) AS remaining_capacity
        FROM room_details rd
        JOIN room_allocation ra ON rd.room_id = ra.room_id
        WHERE ra.exam_id = %(exam_id)s AND rd.room_id = %(room_id)s
    """,
    "who_is_in_room": """
        SELECT s.usn, s.s_name, sa.bench_number, sa.bench_place
        FROM student_allocation sa
        JOIN student s ON sa.usn = s.usn
        WHERE sa.room_id = %(room_id)s AND sa.exam_id = %(exam_id)s
        ORDER BY sa.bench_number
    """,
    "find_exam_in_room": """
        SELECT e.exam_name, e.exam_date, e.slot
        FROM room_allocation ra
        JOIN exam e ON ra.exam_id = e.exam_id
        WHERE ra.room_id = %(room_id)s
    """
}

INTENT_LABELS = list(SQL_TEMPLATES.keys())

In [40]:


def call_local_llama(prompt: str):
    """
    Calls local Llama 3.2 running under Ollama's OpenAI-compatible endpoint.
    Returns string or None.
    """
    try:
        resp = client.chat.completions.create(
            model="llama3.2",
            messages=[
                {"role": "system", "content": "You extract intents and entities. Respond ONLY in JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        print("Local Llama call failed:", e)
        return None
        
def messages_to_prompt(messages):
    """Convert message list to a single prompt."""
    out = []
    for m in messages:
        role = m.get("role","user")
        content = m.get("content","")
        out.append(f"{role.upper()}: {content}")
    return "\n".join(out)

def test_local_llama_connection():
    try:
        resp = client.chat.completions.create(
            model="llama3.2",
            messages=[{"role":"user","content":"ping"}],
            temperature=0
        )
        sample = resp.choices[0].message.content.strip()
        return ("pong" in sample.lower()), sample
    except Exception as e:
        return False, str(e)



In [41]:
# -------------------------
# Intent classifier: regex-first, then local Llama fallback
# -------------------------
def llm_classify_intent_local(text: str) -> Optional[str]:
    sys = (
        "You are an intent classifier for an exam-room allocation system.\n"
        "Return EXACTLY one label from this list (no explanation):\n" + ", ".join(INTENT_LABELS)
    )
    messages = [{"role":"system","content":sys}, {"role":"user","content":text}]
    prompt = messages_to_prompt(messages)
    out = call_local_llama(prompt)

    if not out:
        return None
    label = out.strip().splitlines()[0].strip()
    # cleanup common noise
    label = re.sub(r"[^a-zA-Z0-9_\-]", "", label)
    return label if label in INTENT_LABELS else None

def classify_intent(text: str) -> str:
    t = text.lower()
    # student rules
    if re.search(r"\bwhere\b.*\b(my|i)\b.*\bexam\b", t) or re.search(r"which room.*\b(me|i)\b", t):
        return "student_exam_room"
    if re.search(r"\b(which|what).*\bbench|seat|seat number\b", t):
        return "student_exam_bench"
    if re.search(r"\b(full schedule|timetable|my schedule)\b", t):
        return "student_full_schedule"
    # faculty rules
    if re.search(r"\b(invigilat|where.*i.*invigilat|which room.*i.*invigilat|where.*i.*supervise)\b", t):
        return "faculty_room_duty"
    if re.search(r"\b(list students|who is in my room|students in my room)\b", t):
        return "faculty_student_list"
    if re.search(r"\b(my duties|my invigilation|show all my invigilation)\b", t):
        return "faculty_full_duties"
    # admin/system rules
    if re.search(r"\bfree rooms|unused rooms|which rooms are free\b", t):
        return "unused_rooms"
    if re.search(r"\bnot allocated|unallocated students|students not allocated\b", t):
        return "unallocated_students"
    if re.search(r"\bremaining capacity|capacity left|how many seats left\b", t):
        return "room_remaining_capacity"
    if re.search(r"\bwho is in room|which students are in room\b", t):
        return "who_is_in_room"
    if re.search(r"\bwhich exam is in room|exam in room\b", t):
        return "find_exam_in_room"

    # fallback -> local Llama
    ok_label = llm_classify_intent_local(text)
    if ok_label:
        return ok_label
    # safe default
    return "student_exam_room"


In [42]:
# -------------------------
# Entity extraction: strict regex first, then local Llama fallback
# -------------------------
EXAM_KEYWORDS = ["DBMS","TOC","AIML","CLOUD","POME","OS","CN","SE"]

def llm_extract_entities_local(text: str, intent: str) -> Dict[str, Any]:
    sys = (
        "You are an entity extractor for an exam allocation system.\n"
        "Input: intent label and user question.\n"
        "Output: JSON object with keys (if available): usn, fac_id, room_id, exam_name, exam_date (YYYY-MM-DD), slot, exam_id.\n"
        "Return JSON only, nothing else."
    )
    messages = [{"role":"system","content":sys}, {"role":"user","content":f"Intent: {intent}\nQuestion: {text}"}]
    prompt = messages_to_prompt(messages)
    out = call_local_llama(prompt)

    if not out:
        return {}
    # try parsing JSON in response
    try:
        parsed = json.loads(out)
        if isinstance(parsed, dict):
            return parsed
    except Exception:
        # try to extract JSON substring
        m = re.search(r"\{.*\}", out, re.S)
        if m:
            try:
                parsed = json.loads(m.group(0))
                if isinstance(parsed, dict):
                    return parsed
            except Exception:
                return {}
    return {}

def extract_entities(text: str) -> Dict[str, Any]:
    t = text.strip()
    ent: Dict[str, Any] = {}

    # 1) USN pattern (1RV24IS001 etc)
    m = re.search(r"\b1RV[0-9A-Za-z]{3,}\b", t, re.IGNORECASE)
    if m:
        ent["usn"] = m.group(0).upper()

    # 2) fac_id: FAC001 etc
    m = re.search(r"\b(fac[0-9]{3})\b", t, re.IGNORECASE)
    if m:
        ent["fac_id"] = m.group(1).upper()

    # 3) strict room_id regex (only valid patterns)
    # supports: IS224, IS109, IS112B, IS106A, AS002, AS003, AS004, ISLAB01, ISLAB-01 variations
    m = re.search(r"\b(?:IS|AS)\s?-?\d{3}[A-Z]?\b|\bIS\s?LAB-?\d{2}\b", t, re.IGNORECASE)
    if m:
        rid = m.group(0).upper().replace(" ", "").replace("-", "")
        ent["room_id"] = rid

    # 4) exam_name from known keywords
    for ex in EXAM_KEYWORDS:
        if re.search(r"\b" + re.escape(ex.lower()) + r"\b", t.lower()):
            ent["exam_name"] = ex
            break

    # 5) exam_id explicit mention (e.g., "exam_id 5" or "exam id 5")
    m = re.search(r"exam[_\s]?id\s*[:\-]?\s*(\d+)", t, re.IGNORECASE)
    if m:
        ent["exam_id"] = int(m.group(1))

    # 6) date patterns: yyyy-mm-dd or dd-mm or dd/mm
    m = re.search(r"\b(\d{4}-\d{2}-\d{2})\b", t)
    if m:
        ent["exam_date"] = m.group(1)
    else:
        m2 = re.search(r"\b(\d{1,2}[-/]\d{1,2}(?:[-/]\d{2,4})?)\b", t)
        if m2:
            ent["exam_date_raw"] = m2.group(1)

    # 7) slot HH:MM-HH:MM
    m = re.search(r"(\d{1,2}:\d{2}\s*-\s*\d{1,2}:\d{2})", t)
    if m:
        ent["slot"] = m.group(1)
    else:
        # also accept simple time like "2:30" or "14:30"
        m2 = re.search(r"\b(\d{1,2}:\d{2})\b", t)
        if m2:
            ent["slot_simple"] = m2.group(1)

    return ent

def normalize_date(entity: dict) -> Optional[str]:
    if "exam_date" in entity:
        return entity["exam_date"]
    if "exam_date_raw" in entity:
        raw = entity["exam_date_raw"]
        parts = re.split("[-/]", raw)
        try:
            d = int(parts[0]); m = int(parts[1])
            year = int(parts[2]) if len(parts) == 3 else datetime.now().year
            dt = datetime(year, m, d)
            return dt.strftime("%Y-%m-%d")
        except Exception:
            return None
    return None


In [43]:
# -------------------------
# Fill safe SQL template
# -------------------------
def prepare_query(intent: str, entities: dict) -> Tuple[str, dict]:
    template = SQL_TEMPLATES.get(intent)
    if not template:
        raise ValueError("No template for intent: " + intent)
    params: Dict[str, Any] = {}
    exam_name_filter = ""
    date_slot_filter = ""

    if "usn" in entities:
        params["usn"] = entities["usn"]
    if "fac_id" in entities:
        params["fac_id"] = entities["fac_id"]
    if "room_id" in entities:
        params["room_id"] = entities["room_id"]
    if "exam_name" in entities:
        exam_name_filter = " AND e.exam_name = %(exam_name)s "
        params["exam_name"] = entities["exam_name"]
    if "exam_id" in entities:
        params["exam_id"] = int(entities["exam_id"])

    normalized = normalize_date(entities)
    if normalized:
        params["exam_date"] = normalized
    if "slot" in entities:
        params["slot"] = entities["slot"]
    elif "slot_simple" in entities:
        # convert simple slot to a full slot if you want; for now use as-is
        params["slot"] = entities["slot_simple"]

    if "exam_date" in params and "slot" in params:
        date_slot_filter = " AND e.exam_date = %(exam_date)s AND e.slot = %(slot)s "
    elif "exam_date" in params:
        date_slot_filter = " AND e.exam_date = %(exam_date)s "

    query = template.format(exam_name_filter=exam_name_filter, date_slot_filter=date_slot_filter)
    return query, params

# -------------------------
# Main pipeline with LLM entity fallback and safe defaults
# -------------------------
def nl_to_sql_and_run(nl_text: str, execute: bool = True) -> Dict[str, Any]:
    text = nl_text.strip()
    intent = classify_intent(text)
    entities = extract_entities(text)

    # Decide whether entity fallback is needed
    need_entity_fallback = False
    if intent.startswith("student"):
        if "usn" not in entities:
            need_entity_fallback = True
    if intent.startswith("faculty"):
        # require fac_id OR (exam_date+slot) to locate duty
        if "fac_id" not in entities and not ("exam_date" in entities or "slot" in entities or "exam_date_raw" in entities):
            need_entity_fallback = True
    if intent in ("unallocated_students", "unused_rooms", "find_exam_in_room", "who_is_in_room", "room_remaining_capacity"):
        # these need specific keys
        if intent == "unallocated_students" and "exam_id" not in entities:
            need_entity_fallback = True
        if intent == "unused_rooms" and not ("exam_date" in entities or "exam_date_raw" in entities or "slot" in entities or "slot_simple" in entities):
            need_entity_fallback = True
        if intent in ("who_is_in_room", "find_exam_in_room", "room_remaining_capacity") and "room_id" not in entities:
            need_entity_fallback = True

    llm_entities = {}
    if need_entity_fallback and LOCAL_LLAMA["enabled"]:
        print("Entity fallback: calling local Llama 3.2...")
        llm_entities = llm_extract_entities_local(text, intent)
        # merge
        for k,v in llm_entities.items():
            if k not in entities:
                entities[k] = v

    # If LLM unavailable or returned nothing and still missing critical items, use safe heuristics
    if need_entity_fallback and not llm_entities:
        print("Local Llama unavailable or returned nothing — applying safe heuristics/fallbacks.")
        # faculty heuristics: if text contains "afternoon"/"morning" pick slot
        low = text.lower()
        if intent == "faculty_room_duty":
            if "fac_id" in entities and "slot" not in entities:
                if "afternoon" in low or "2:30" in low or "14:30" in low:
                    entities.setdefault("slot", "14:30-16:30")
                else:
                    entities.setdefault("slot", "09:30-11:30")
        if intent == "unused_rooms":
            entities.setdefault("slot", "09:30-11:30")
            # attempt to normalize date from "today/tomorrow"
            if "today" in low:
                entities.setdefault("exam_date", datetime.now().strftime("%Y-%m-%d"))
            elif "tomorrow" in low:
                entities.setdefault("exam_date", (datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d"))
        if intent == "unallocated_students":
            # try to extract exam_id again via loose regex
            m = re.search(r"\bexam[_\s]?id\s*(\d+)\b", low)
            if m:
                entities["exam_id"] = int(m.group(1))

    # normalize dates in entities
    if "exam_date_raw" in entities and "exam_date" not in entities:
        nd = normalize_date(entities)
        if nd:
            entities["exam_date"] = nd

    # prepare SQL
    try:
        sql, params = prepare_query(intent, entities)
    except Exception as e:
        return {"error": f"Prepare query failed: {e}", "intent": intent, "entities": entities}

    # execute
    if execute:
        try:
            rows = run_sql_fetchall(sql, params)
            return {"intent": intent, "entities": entities, "sql": sql, "params": params, "rows": rows}
        except Exception as e:
            return {"error": f"SQL execution error: {e}", "intent": intent, "entities": entities, "sql": sql, "params": params}
    else:
        return {"intent": intent, "entities": entities, "sql": sql, "params": params}

In [44]:

# -------------------------
# Quick tests
# -------------------------
if __name__ == "__main__":
    # test Llama local connection (user asked for test)
    ok, sample = test_local_llama_connection()
    print("Local Llama connected:", ok, "sample:", sample)

    examples = [
        "Where is my exam DBMS? USN 1RV24IS001",
        "Which bench am I sitting on for CLOUD? 1RV24IS135",
        "FAC002 where do i invigilate on 2025-10-26 at 14:30",
        "Which rooms are free on 2025-10-26 at 14:30?",
        "How many students are not allocated for exam_id 5?",
        "Who is in room IS224 for exam id 5?",
        "Where should I go tomorrow afternoon? I am FAC007"
    ]

    for ex in examples:
        print("\n---")
        print("NL:", ex)
        out = nl_to_sql_and_run(ex, execute=True)
        print("Intent:", out.get("intent"))
        print("Entities:", out.get("entities"))
        print("SQL (preview):", (out.get("sql") or "").strip()[:400])
        print("Params:", out.get("params"))
        if out.get("rows") is not None:
            print("Rows (sample):", out.get("rows")[:5])
        else:
            print("Error:", out.get("error"))

Local Llama connected: False sample: *ping*

Response time: 200 ms

---
NL: Where is my exam DBMS? USN 1RV24IS001
Intent: student_exam_room
Entities: {'usn': '1RV24IS001', 'exam_name': 'DBMS'}
SQL (preview): SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, e.exam_date, e.exam_name
        FROM student_allocation sa
        JOIN exam e ON sa.exam_id = e.exam_id
        WHERE sa.usn = %(usn)s
         AND e.exam_name = %(exam_name)s
Params: {'usn': '1RV24IS001', 'exam_name': 'DBMS'}
Rows (sample): [{'room_id': 'IS112B', 'bench_number': 17, 'bench_place': 'middle', 'time_slot': '09:30-11:30', 'exam_date': datetime.date(2025, 10, 25), 'exam_name': 'DBMS'}]

---
NL: Which bench am I sitting on for CLOUD? 1RV24IS135
Intent: student_exam_bench
Entities: {'usn': '1RV24IS135', 'exam_name': 'CLOUD'}
SQL (preview): SELECT sa.room_id, sa.bench_number, sa.bench_place
        FROM student_allocation sa
        JOIN exam e ON sa.exam_id = e.exam_id
        WHERE sa.usn = %(usn)s
    

In [18]:
# ------------------------------
# TEST LOCAL LLAMA CONNECTION
# ------------------------------
print("Testing connection to local Llama 3.2...")

try:
    resp = client.chat.completions.create(
        model="llama3.2",
        messages=[{"role": "user", "content": "ping"}],
        temperature=0
    )
    print("Local Llama connected:", True, "sample:", resp.choices[0].message.content)
except Exception as e:
    print("Local Llama connected:", False)
    print("Error:", e)


Testing connection to local Llama 3.2...
Local Llama connected: True sample: *ping*

Response time: 200 ms


In [52]:
# NL2SQL pipeline - Full corrected notebook
# Uses local Ollama (OpenAI-compatible) llama3.2 for intent+entity fallback
# Strict JSON output required from LLM. Safe SQL template engine for your schema.

import os
import re
import json
import mysql.connector
from datetime import datetime, timedelta
from typing import Dict, Any, Tuple, Optional
from openai import OpenAI

# -------------------------
# Config - update as needed
# -------------------------
MYSQL_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "naraayana",   
    "database": "ml_db_collection"
}

# Path to uploaded PDF (from conversation)
UPLOADED_PDF_PATH = "/mnt/data/3rd sem Lab Batch List.pdf"

# OpenAI-compatible local Ollama settings via env
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # should be 'ollama'
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")  # e.g. http://localhost:11434/v1

client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

# -------------------------
# SQL templates (whitelisted)
# -------------------------
SQL_TEMPLATES = {
    "student_exam_room": (
        "SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, e.exam_date, e.exam_name\n"
        "FROM student_allocation sa\n"
        "JOIN exam e ON sa.exam_id = e.exam_id\n"
        "WHERE sa.usn = %(usn)s {exam_name_filter}"
    ),
    "student_exam_bench": (
        "SELECT sa.room_id, sa.bench_number, sa.bench_place\n"
        "FROM student_allocation sa\n"
        "JOIN exam e ON sa.exam_id = e.exam_id\n"
        "WHERE sa.usn = %(usn)s {exam_name_filter}"
    ),
    "student_full_schedule": (
        "SELECT e.exam_name, e.exam_date, e.slot, sa.room_id, sa.bench_number, sa.bench_place\n"
        "FROM student_allocation sa\n"
        "JOIN exam e ON sa.exam_id = e.exam_id\n"
        "WHERE sa.usn = %(usn)s\nORDER BY e.exam_date, e.slot"
    ),
    "faculty_invigilation": (
        "SELECT ra.room_id, rd.capacity, e.exam_name, e.exam_date, e.slot, ra.no_students\n"
        "FROM room_allocation ra\n"
        "JOIN exam e ON ra.exam_id = e.exam_id\n"
        "JOIN room_details rd ON rd.room_id = ra.room_id\n"
        "WHERE ra.fac_id = %(fac_id)s {date_slot_filter}"
    ),
    "faculty_student_list": (
        "SELECT sa.usn, s.s_name, sa.bench_number, sa.bench_place\n"
        "FROM student_allocation sa\n"
        "JOIN student s ON sa.usn = s.usn\n"
        "WHERE sa.fac_id = %(fac_id)s AND sa.exam_id = %(exam_id)s\n"
        "ORDER BY sa.bench_number, sa.bench_place"
    ),
    "unused_rooms": (
        "SELECT rd.room_id, rd.capacity\n"
        "FROM room_details rd\n"
        "WHERE rd.room_id NOT IN (\n"
        "  SELECT room_id FROM room_allocation ra\n"
        "  JOIN exam e ON ra.exam_id = e.exam_id\n"
        "  WHERE e.exam_date = %(exam_date)s AND e.slot = %(slot)s\n"
        ")"
    ),
    "unallocated_students": (
        "SELECT (SELECT COUNT(*) FROM student) - (SELECT COUNT(*) FROM student_allocation WHERE exam_id = %(exam_id)s) AS not_allocated_students"
    ),
    "room_remaining_capacity": (
        "SELECT rd.room_id, rd.capacity, ra.no_students, (rd.capacity - ra.no_students) AS remaining_capacity\n"
        "FROM room_details rd\n"
        "JOIN room_allocation ra ON rd.room_id = ra.room_id\n"
        "WHERE ra.exam_id = %(exam_id)s AND rd.room_id = %(room_id)s"
    ),
    "who_is_in_room": (
        "SELECT s.usn, s.s_name, sa.bench_number, sa.bench_place\n"
        "FROM student_allocation sa\n"
        "JOIN student s ON sa.usn = s.usn\n"
        "WHERE sa.room_id = %(room_id)s AND sa.exam_id = %(exam_id)s\nORDER BY sa.bench_number"
    )
}

# Allowed intent labels
INTENT_LABELS = [
    "student_exam_room",
    "student_exam_bench",
    "student_full_schedule",
    "faculty_invigilation",
    "faculty_student_list",
    "unused_rooms",
    "unallocated_students",
    "room_remaining_capacity",
    "who_is_in_room"
]

# -------------------------
# Strict Llama system prompt (forces JSON-only)
# -------------------------
LLAMA_SYSTEM_PROMPT = r'''
You are a strict intent+entity extractor.
Return ONLY valid JSON with this structure:
{
  "intent": "one_of_allowed_labels",
  "entities": { ... }
}
Allowed intents: %s
Allowed entity keys: usn, fac_id, room_id, exam_name, exam_id, exam_date (YYYY-MM-DD), slot, slot_simple
RULES:
- Do NOT invent values. Only include keys explicitly present in the user message.
- Do NOT output any extra keys.
- If a value is not present, omit the key.
- Return JSON ONLY, no explanation.
''' % (", ".join(INTENT_LABELS))

# -------------------------
# DB helpers
# -------------------------
def db_connect():
    return mysql.connector.connect(**MYSQL_CONFIG)

def run_sql_fetchall(query: str, params: dict = None):
    conn = db_connect()
    cur = conn.cursor(dictionary=True)
    cur.execute(query, params or {})
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return rows

def run_sql_execute(query: str, params: dict = None):
    conn = db_connect()
    cur = conn.cursor()
    cur.execute(query, params or {})
    conn.commit()
    cur.close()
    conn.close()

# -------------------------
# OpenAI-compatible local Ollama call (uses openai.OpenAI client)
# -------------------------
def call_local_llama(prompt: str) -> Optional[dict]:
    """Calls local Ollama via OpenAI-compatible client. Returns parsed JSON dict or None."""
    try:
        resp = client.chat.completions.create(
            model="llama3.2",
            messages=[
                {"role": "system", "content": LLAMA_SYSTEM_PROMPT},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        txt = resp.choices[0].message.content.strip()
        try:
            data = json.loads(txt)
            # sanitize: only allow keys we expect
            if not isinstance(data, dict) or "intent" not in data or "entities" not in data:
                print("Llama returned JSON but missing keys:", txt)
                return None
            return data
        except Exception:
            print("Llama returned non-JSON:\n", txt)
            return None
    except Exception as e:
        print("Local Llama call failed:", e)
        return None

# -------------------------
# Utilities
# -------------------------
FACULTY_PATTERN = re.compile(r"\bFAC\d{3}\b", re.IGNORECASE)
USN_PATTERN = re.compile(r"\b1RV[0-9A-Za-z]{3,}\b", re.IGNORECASE)
ROOM_PATTERN = re.compile(r"\b(?:IS|AS)\s?-?\d{3}[A-Z]?\b|\bIS\s?LAB-?\d{2}\b", re.IGNORECASE)
EXAM_KEYWORDS = ["DBMS","TOC","AIML","CLOUD","POME","OS","CN","SE"]

def messages_to_prompt(messages: list) -> str:
    out = []
    for m in messages:
        role = m.get("role", "user")
        content = m.get("content", "")
        out.append(f"{role.upper()}: {content}")
    return "\n".join(out)

# -------------------------
# Intent classifier (regex-first) with faculty priority
# -------------------------
def classify_intent(nl: str) -> str:
    t = nl.lower()
    # faculty explicit
    if FACULTY_PATTERN.search(nl):
        return "faculty_invigilation"
    # student rules
    if re.search(r"\bwhere\b.*\b(my|i)\b.*\bexam\b", t) or re.search(r"which room.*\b(me|i)\b", t):
        return "student_exam_room"
    if re.search(r"\b(which|what).*(bench|seat|seat number)\b", t):
        return "student_exam_bench"
    if re.search(r"\b(full schedule|timetable|my schedule)\b", t):
        return "student_full_schedule"
    # admin/system
    if re.search(r"\bfree rooms|unused rooms|which rooms are free\b", t):
        return "unused_rooms"
    if re.search(r"\bnot allocated|unallocated students|students not allocated\b", t):
        return "unallocated_students"
    if re.search(r"\bremaining capacity|capacity left|how many seats left\b", t):
        return "room_remaining_capacity"
    if re.search(r"\bwho is in room|which students are in room\b", t):
        return "who_is_in_room"
    if re.search(r"\bwhich exam is in room|exam in room\b", t):
        return "who_is_in_room"
    # fallback default student
    return "student_exam_room"

# -------------------------
# Entity extraction (regex-first)
# -------------------------

def extract_entities(nl: str) -> Dict[str, Any]:
    ent: Dict[str, Any] = {}
    nl_lower = nl.lower()
    # usn
    m = USN_PATTERN.search(nl)
    if m:
        ent["usn"] = m.group(0).upper()
    # fac
    m = FACULTY_PATTERN.search(nl)
    if m:
        ent["fac_id"] = m.group(0).upper()
    # room
    m = ROOM_PATTERN.search(nl)
    if m:
        ent["room_id"] = m.group(0).upper().replace(" ", "").replace("-", "")
    # exam name
    for ex in EXAM_KEYWORDS:
        if re.search(r"\b" + re.escape(ex.lower()) + r"\b", nl_lower):
            ent["exam_name"] = ex
            break
    # exam_id
    m = re.search(r"exam[_\s]?id\s*[:\-]?\s*(\d+)", nl_lower)
    if m:
        ent["exam_id"] = int(m.group(1))
    # date
    m = re.search(r"(\d{4}-\d{2}-\d{2})", nl)
    if m:
        ent["exam_date"] = m.group(1)
    else:
        m2 = re.search(r"(\d{1,2}[-/]\d{1,2}(?:[-/]\d{2,4})?)", nl)
        if m2:
            ent["exam_date_raw"] = m2.group(1)
    # slot
    m = re.search(r"(\d{1,2}:\d{2}\s*-\s*\d{1,2}:\d{2})", nl)
    if m:
        ent["slot"] = m.group(1)
    else:
        m2 = re.search(r"\b(\d{1,2}:\d{2})\b", nl)
        if m2:
            ent["slot_simple"] = m2.group(1)
    return ent

def normalize_date(entity: dict) -> Optional[str]:
    if "exam_date" in entity:
        return entity["exam_date"]
    if "exam_date_raw" in entity:
        raw = entity["exam_date_raw"]
        parts = re.split("[-/]", raw)
        try:
            d = int(parts[0]); m = int(parts[1])
            year = int(parts[2]) if len(parts) == 3 else datetime.now().year
            dt = datetime(year, m, d)
            return dt.strftime("%Y-%m-%d")
        except Exception:
            return None
    return None

# -------------------------
# Prepare SQL safely
# -------------------------

def prepare_query(intent: str, entities: dict) -> Tuple[str, dict]:
    template = SQL_TEMPLATES.get(intent)
    if not template:
        raise ValueError("No template for intent: " + intent)
    params: Dict[str, Any] = {}
    exam_name_filter = ""
    date_slot_filter = ""
    if "usn" in entities:
        params["usn"] = entities["usn"]
    if "fac_id" in entities:
        params["fac_id"] = entities["fac_id"]
    if "room_id" in entities:
        params["room_id"] = entities["room_id"]
    if "exam_name" in entities:
        exam_name_filter = " AND e.exam_name = %(exam_name)s "
        params["exam_name"] = entities["exam_name"]
    if "exam_id" in entities:
        params["exam_id"] = int(entities["exam_id"])
    normalized = normalize_date(entities)
    if normalized:
        params["exam_date"] = normalized
    if "slot" in entities:
        params["slot"] = entities["slot"]
    elif "slot_simple" in entities:
        params["slot"] = entities["slot_simple"]
    if "exam_date" in params and "slot" in params:
        date_slot_filter = " AND e.exam_date = %(exam_date)s AND e.slot = %(slot)s "
    elif "exam_date" in params:
        date_slot_filter = " AND e.exam_date = %(exam_date)s "
    query = template.format(exam_name_filter=exam_name_filter, date_slot_filter=date_slot_filter)
    return query, params

# -------------------------
# Main NL->SQL pipeline
# -------------------------

def nl_to_sql_and_run(nl_text: str, execute: bool = True) -> Dict[str, Any]:
    text = nl_text.strip()
    intent = classify_intent(text)
    entities = extract_entities(text)

    # decide if entity fallback needed
    need_entity_fallback = False
    if intent.startswith("student"):
        if "usn" not in entities:
            need_entity_fallback = True
    if intent.startswith("faculty"):
        if "fac_id" not in entities and not ("exam_date" in entities or "slot" in entities or "exam_date_raw" in entities):
            need_entity_fallback = True
    if intent in ("unallocated_students", "unused_rooms", "who_is_in_room", "room_remaining_capacity"):
        if intent == "unallocated_students" and "exam_id" not in entities:
            need_entity_fallback = True
        if intent == "unused_rooms" and not ("exam_date" in entities or "exam_date_raw" in entities or "slot" in entities or "slot_simple" in entities):
            need_entity_fallback = True
        if intent in ("who_is_in_room", "room_remaining_capacity") and "room_id" not in entities:
            need_entity_fallback = True

    llm_json = None
    if need_entity_fallback:
        prompt = f"Extract only JSON for intent+entities from this user message:\n{text}"
        llm_json = call_local_llama(prompt)
        if llm_json:
            # merge but ONLY allowed keys
            ents = llm_json.get("entities", {}) if isinstance(llm_json, dict) else {}
            for k, v in ents.items():
                if k in {"usn","fac_id","room_id","exam_name","exam_id","exam_date","slot","slot_simple"} and v:
                    entities.setdefault(k, v)

    # sanitize empty strings
    for k in list(entities.keys()):
        if entities[k] == "" or entities[k] is None:
            del entities[k]

    # fallback heuristics if LLM unavailable
    if need_entity_fallback and not llm_json:
        low = text.lower()
        if intent == "faculty_invigilation":
            if "slot" not in entities:
                if "afternoon" in low or "14:30" in low or "2:30" in low:
                    entities.setdefault("slot", "14:30-16:30")
                else:
                    entities.setdefault("slot", "09:30-11:30")
        if intent == "unused_rooms":
            entities.setdefault("slot", "09:30-11:30")
            if "today" in low:
                entities.setdefault("exam_date", datetime.now().strftime("%Y-%m-%d"))
            elif "tomorrow" in low:
                entities.setdefault("exam_date", (datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d"))

    # normalize date
    if "exam_date_raw" in entities and "exam_date" not in entities:
        nd = normalize_date(entities)
        if nd:
            entities["exam_date"] = nd
    # auto-fill slot if user didn't specify and intent is unused_rooms
    if intent == "unused_rooms" and "slot" not in entities:
        # system knows only two slots
        entities["slot"] = "09:30-11:30" if "morning" in nl_text.lower() else None
    
        # no morning/afternoon text → return both
        if entities["slot"] is None:
            return {
                "intent": intent,
                "entities": entities,
                "two_queries": [
                    {
                        "slot": "09:30-11:30",
                        "sql": SQL_TEMPLATES["unused_rooms"],
                        "params": {
                            "exam_date": entities["exam_date"],
                            "slot": "09:30-11:30"
                        },
                        "rows": run_sql_fetchall(
                            SQL_TEMPLATES["unused_rooms"],
                            {"exam_date": entities["exam_date"], "slot": "09:30-11:30"}
                        )
                    },
                    {
                        "slot": "14:30-16:30",
                        "sql": SQL_TEMPLATES["unused_rooms"],
                        "params": {
                            "exam_date": entities["exam_date"],
                            "slot": "14:30-16:30"
                        },
                        "rows": run_sql_fetchall(
                            SQL_TEMPLATES["unused_rooms"],
                            {"exam_date": entities["exam_date"], "slot": "14:30-16:30"}
                        )
                    }
                ]
            }

    try:
        sql, params = prepare_query(intent, entities)
    except Exception as e:
        return {"error": f"Prepare query failed: {e}", "intent": intent, "entities": entities}

    if execute:
        try:
            rows = run_sql_fetchall(sql, params)
            return {"intent": intent, "entities": entities, "sql": sql, "params": params, "rows": rows}
        except Exception as e:
            return {"error": f"SQL execution error: {e}", "intent": intent, "entities": entities, "sql": sql, "params": params}
    else:
        return {"intent": intent, "entities": entities, "sql": sql, "params": params}

# -------------------------
# Test harness
# -------------------------
if __name__ == "__main__":
    ok, sample = False, None
    try:
        # quick ping test
        resp = client.chat.completions.create(model="llama3.2", messages=[{"role":"user","content":"ping"}], temperature=0)
        sample = resp.choices[0].message.content
        ok = "pong" in sample.lower() or sample.strip().lower().startswith("*ping*") or "ping" in sample.lower()
    except Exception as e:
        ok = False
        sample = str(e)
    print("Local Llama connected:", ok, "sample:", sample)

    examples = [
        "Where is my exam DBMS? USN 1RV24IS001",
        "Which bench am I sitting on for CLOUD? 1RV24IS135",
        "FAC002 where do i invigilate on 2025-10-26 at 14:30",
        "Which rooms are free on 2025-10-26 at 14:30?",
        "How many students are not allocated for exam_id 5?",
        "Who is in room IS224 for exam id 5?",
        "Where should I go tomorrow afternoon? I am FAC007",
        "iam 1RV23IS007, tell my all 5 exam locations",
        "which rooms are empty on 2025-10-26"
    ]

    for ex in examples:
        print('\n---')
        print('NL:', ex)
        out = nl_to_sql_and_run(ex, execute=True)
        print('Intent:', out.get('intent'))
        print('Entities:', out.get('entities'))
        print('SQL (preview):', (out.get('sql') or '').strip()[:400])
        print('Params:', out.get('params'))
        if out.get('rows') is not None:
            print('Rows (sample):', out.get('rows')[:5])
        else:
            print('Error:', out.get('error'))


Local Llama connected: True sample: *ping*

Response time: 200 ms

---
NL: Where is my exam DBMS? USN 1RV24IS001
Intent: student_exam_room
Entities: {'usn': '1RV24IS001', 'exam_name': 'DBMS'}
SQL (preview): SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, e.exam_date, e.exam_name
FROM student_allocation sa
JOIN exam e ON sa.exam_id = e.exam_id
WHERE sa.usn = %(usn)s  AND e.exam_name = %(exam_name)s
Params: {'usn': '1RV24IS001', 'exam_name': 'DBMS'}
Rows (sample): [{'room_id': 'IS112B', 'bench_number': 17, 'bench_place': 'middle', 'time_slot': '09:30-11:30', 'exam_date': datetime.date(2025, 10, 25), 'exam_name': 'DBMS'}]

---
NL: Which bench am I sitting on for CLOUD? 1RV24IS135
Intent: student_exam_bench
Entities: {'usn': '1RV24IS135', 'exam_name': 'CLOUD'}
SQL (preview): SELECT sa.room_id, sa.bench_number, sa.bench_place
FROM student_allocation sa
JOIN exam e ON sa.exam_id = e.exam_id
WHERE sa.usn = %(usn)s  AND e.exam_name = %(exam_name)s
Params: {'usn': '1RV24IS135'

In [3]:
# NL2SQL pipeline - Full corrected notebook (OVERWRITE v2)
# Uses local Ollama (OpenAI-compatible) llama3.2 for intent+entity fallback
# Enforces strict JSON output from LLM and robust sanitization. Safe SQL templates.

import os
import re
import json
import mysql.connector
from datetime import datetime, timedelta
from typing import Dict, Any, Tuple, Optional
from openai import OpenAI

# -------------------------
# Config - update as needed (env vars used if present)
# -------------------------
MYSQL_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "naraayana",   
    "database": "ml_db_collection"
}

# Path to uploaded PDF (sample)
UPLOADED_PDF_PATH = "/mnt/data/3rd sem Lab Batch List.pdf"

import requests


# OpenAI-compatible local Ollama settings via env
OPENAI_API_KEY = "ollama"  # should be 'ollama'
OPENAI_BASE_URL = "http://localhost:11434/v1"  # e.g. http://localhost:11434/v1

client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)



# -------------------------
# SQL templates (whitelisted)
# -------------------------
SQL_TEMPLATES = {
        "student_exam_room": (
        "SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, "
        "e.exam_date, e.exam_name\n"
        "FROM student_allocation sa\n"
        "JOIN exam e ON sa.exam_id = e.exam_id\n"
        "WHERE sa.usn = %(usn)s "
        "{exam_name_filter} "
        "{date_slot_filter}"
    ),
    "student_exam_bench": (
        "SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, "
        "e.exam_date, e.exam_name\n"
        "FROM student_allocation sa\n"
        "JOIN exam e ON sa.exam_id = e.exam_id\n"
        "WHERE sa.usn = %(usn)s "
        "{exam_name_filter} "
        "{date_slot_filter}"
    ),
    "student_full_schedule": (
        "SELECT e.exam_name, e.exam_date, e.slot, sa.room_id, sa.bench_number, sa.bench_place\n"
        "FROM student_allocation sa\n"
        "JOIN exam e ON sa.exam_id = e.exam_id\n"
        "WHERE sa.usn = %(usn)s\nORDER BY e.exam_date, e.slot"
    ),
    "faculty_invigilation": (
        "SELECT ra.room_id, rd.capacity, e.exam_name, e.exam_date, e.slot, ra.no_students\n"
        "FROM room_allocation ra\n"
        "JOIN exam e ON ra.exam_id = e.exam_id\n"
        "JOIN room_details rd ON rd.room_id = ra.room_id\n"
        "WHERE ra.fac_id = %(fac_id)s {date_slot_filter}"
    ),
    "faculty_student_list": (
        "SELECT sa.usn, s.s_name, sa.bench_number, sa.bench_place\n"
        "FROM student_allocation sa\n"
        "JOIN student s ON sa.usn = s.usn\n"
        "WHERE sa.fac_id = %(fac_id)s AND sa.exam_id = %(exam_id)s\n"
        "ORDER BY sa.bench_number, sa.bench_place"
    ),
   
    "unallocated_students": (
        "SELECT (SELECT COUNT(*) FROM student) - (SELECT COUNT(*) FROM student_allocation WHERE exam_id = %(exam_id)s) AS not_allocated_students"
    ),
    "room_remaining_capacity": (
        "SELECT rd.room_id, rd.capacity, ra.no_students, (rd.capacity - ra.no_students) AS remaining_capacity\n"
        "FROM room_details rd\n"
        "JOIN room_allocation ra ON rd.room_id = ra.room_id\n"
        "WHERE ra.exam_id = %(exam_id)s AND rd.room_id = %(room_id)s"
    ),
    "who_is_in_room": (
        "SELECT s.usn, s.s_name, sa.bench_number, sa.bench_place\n"
        "FROM student_allocation sa\n"
        "JOIN student s ON sa.usn = s.usn\n"
        "WHERE sa.room_id = %(room_id)s AND sa.exam_id = %(exam_id)s\nORDER BY sa.bench_number"
    )
}

INTENT_LABELS = list(SQL_TEMPLATES.keys())

# -------------------------
# Strict Llama system prompt (forces JSON-only)
# -------------------------
LLAMA_SYSTEM_PROMPT = r'''
You are a strict intent+entity extractor for an exam-room system.
Return ONLY valid JSON with this structure exactly:
{
  "intent": "one_of_allowed_labels",
  "entities": { ... }
}
Allowed intents: %s
Allowed entity keys: usn, fac_id, room_id, exam_name, exam_id, exam_date (YYYY-MM-DD), slot, slot_simple
RULES:
- Do NOT invent values. Only include keys explicitly present in the user message.
- If a value is not present, omit the key.
- Return JSON ONLY, no explanation.
''' % (", ".join(INTENT_LABELS))

# -------------------------
# DB helpers
# -------------------------

def db_connect():
    return mysql.connector.connect(**MYSQL_CONFIG)

def run_sql_fetchall(query: str, params: dict = None):
    conn = db_connect()
    cur = conn.cursor(dictionary=True)
    cur.execute(query, params or {})
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return rows

def run_sql_execute(query: str, params: dict = None):
    conn = db_connect()
    cur = conn.cursor()
    cur.execute(query, params or {})
    conn.commit()
    cur.close()
    conn.close()

# -------------------------
# OpenAI-compatible Ollama call (uses openai.OpenAI client)
# Returns parsed JSON or None
# -------------------------

def call_local_llama(prompt: str) -> Optional[Dict[str, Any]]:
    try:
        resp = client.chat.completions.create(
            model="llama3.2",
            messages=[
                {"role": "system", "content": LLAMA_SYSTEM_PROMPT},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        txt = resp.choices[0].message.content.strip()
        try:
            data = json.loads(txt)
            # basic validation: must have intent and entities
            if not isinstance(data, dict) or "intent" not in data or "entities" not in data:
                print("Llama returned JSON but missing keys:\n", txt)
                return None
            return data
        except Exception:
            print("Llama returned non-JSON:\n", txt)
            return None
    except Exception as e:
        print("Local Llama call failed:", e)
        return None

# -------------------------
# Utilities & patterns
# -------------------------
FACULTY_PATTERN = re.compile(r"\bFAC\d{3}\b", re.IGNORECASE)
USN_PATTERN = re.compile(r"\b1RV[0-9A-Za-z]{3,}\b", re.IGNORECASE)
ROOM_PATTERN = re.compile(r"\b(?:IS|AS)\s?-?\d{2,3}[A-Z]?\b|\bIS\s?LAB-?\d{1,2}\b", re.IGNORECASE)
EXAM_KEYWORDS = ["DBMS","TOC","AIML","CLOUD","POME","OS","CN","SE"]

INVALID_ENTITY_VALUES = {"", " ", "none", "null", "[empty]", "n/a", "na"}

def messages_to_prompt(messages: list) -> str:
    out = []
    for m in messages:
        role = m.get("role", "user")
        content = m.get("content", "")
        out.append(f"{role.upper()}: {content}")
    return "\n".join(out)

# -------------------------
# Intent classifier (regex-first) with faculty priority
# -------------------------

def classify_intent(nl: str) -> str:
    t = nl.lower()

    # faculty always highest priority
    if FACULTY_PATTERN.search(nl):
        return "faculty_invigilation"

    # free / empty rooms
    if re.search(r"\b(empty|free|unused)\b", t) and "room" in t:
        return "unused_rooms"
    if re.search(r"which rooms are (free|empty|available)", t):
        return "unused_rooms"

    # student asking where *their* exam is
    if re.search(r"\bwhere\b.*\b(my|i)\b.*\bexam\b", t):
        return "student_exam_room"

    # avoid triggering on 'which rooms'
    if re.search(r"\bwhich room\b(?!s)", t) and re.search(r"\b(me|i)\b", t):
        return "student_exam_room"

    # student bench query
    if re.search(r"\b(which|what).*(bench|seat|seat number)\b", t):
        return "student_exam_bench"

    # full schedule
    if re.search(r"\b(full schedule|timetable|my schedule)\b", t):
        return "student_full_schedule"

    # unallocated students
    if re.search(r"\bnot allocated|unallocated students|students not allocated\b", t):
        return "unallocated_students"

    # room capacity left
    if re.search(r"\bremaining capacity|capacity left|how many seats left\b", t):
        return "room_remaining_capacity"

    # who is in room
    if re.search(r"\bwho is in room|which students are in room\b", t):
        return "who_is_in_room"

    # default fallback
    return "student_exam_room"

# -------------------------
# Entity extraction (regex-first)
# -------------------------

def extract_entities(nl: str) -> Dict[str, Any]:
    ent: Dict[str, Any] = {}
    nl_lower = nl.lower()
    m = USN_PATTERN.search(nl)
    if m:
        ent["usn"] = m.group(0).upper()
    m = FACULTY_PATTERN.search(nl)
    if m:
        ent["fac_id"] = m.group(0).upper()
    m = ROOM_PATTERN.search(nl)
    if m:
        ent["room_id"] = m.group(0).upper().replace(" ", "").replace("-", "")
    for ex in EXAM_KEYWORDS:
        if re.search(r"\b" + re.escape(ex.lower()) + r"\b", nl_lower):
            ent["exam_name"] = ex
            break
    m = re.search(r"exam[_\s]?id\s*[:\-]?\s*(\d+)", nl_lower)
    if m:
        ent["exam_id"] = int(m.group(1))
    m = re.search(r"(\d{4}-\d{2}-\d{2})", nl)
    if m:
        ent["exam_date"] = m.group(1)
    else:
        m2 = re.search(r"(\d{1,2}[-/]\d{1,2}(?:[-/]\d{2,4})?)", nl)
        if m2:
            ent["exam_date_raw"] = m2.group(1)
    m = re.search(r"(\d{1,2}:\d{2}\s*-\s*\d{1,2}:\d{2})", nl)
    if m:
        ent["slot"] = m.group(1)
    else:
        m2 = re.search(r"\b(\d{1,2}:\d{2})\b", nl)
        if m2:
            ent["slot_simple"] = m2.group(1)
    return ent


def normalize_date(entity: dict) -> Optional[str]:
    if "exam_date" in entity:
        return entity["exam_date"]
    if "exam_date_raw" in entity:
        raw = entity["exam_date_raw"]
        parts = re.split("[-/]", raw)
        try:
            d = int(parts[0]); m = int(parts[1])
            year = int(parts[2]) if len(parts) == 3 else datetime.now().year
            dt = datetime(year, m, d)
            return dt.strftime("%Y-%m-%d")
        except Exception:
            return None
    return None

# -------------------------
# Sanitize Llama JSON (prevents hallucinated keys/values)
# -------------------------

def sanitize_llama_output(js: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    if not js or "intent" not in js or "entities" not in js:
        return None
    intent = js["intent"]
    entities = js["entities"] if isinstance(js["entities"], dict) else {}
    # remove invalid entity values
    clean = {}
    for k, v in entities.items():
        sval = str(v).strip()
        if sval.lower() in INVALID_ENTITY_VALUES:
            continue
        clean[k] = v
    js["entities"] = clean
    # Keep intents even if entities missing for certain intents
    if intent == "unused_rooms":
        return js
    if intent == "faculty_invigilation":
        return js
    # For student intents require usn present
    if intent in ("student_exam_bench", "student_exam_room"):
        if "usn" in clean:
            return js
        else:
            return None
    # default: return js
    return js

# -------------------------
# Prepare SQL safely
# -------------------------

def prepare_query(intent: str, entities: dict) -> Tuple[str, dict]:
    # slot normalization
    if "slot" in entities:
        s = entities["slot"].strip()
        # normalize variants: "9:30-11:30", "09:30 - 11:30", "930-1130"
        if "9:30" in s and "11:30" in s:
            entities["slot"] = "09:30-11:30"
        if "2:30" in s or "14:30" in s:
            entities["slot"] = "14:30-16:30"

    # -----------------------------------------
    # SPECIAL CASE: unused_rooms (standalone)
    # -----------------------------------------
    if intent == "unused_rooms":
        params = {}

        normalized = normalize_date(entities)
        if normalized:
            params["exam_date"] = normalized
        else:
            params["exam_date"] = entities.get("exam_date")

        slot = entities.get("slot") or entities.get("slot_simple")

        # Case 1 → only date given → show rooms empty whole day
        if slot is None:
            sql = """
            SELECT rd.room_id, rd.capacity
            FROM room_details rd
            WHERE rd.room_id NOT IN (
                SELECT room_id FROM room_allocation ra
                JOIN exam e ON ra.exam_id = e.exam_id
                WHERE e.exam_date = %(exam_date)s
            );
            """
            return sql, params

        # Case 2 → date + slot
        params["slot"] = slot
        sql = """
        SELECT rd.room_id, rd.capacity
        FROM room_details rd
        WHERE rd.room_id NOT IN (
            SELECT room_id FROM room_allocation ra
            JOIN exam e ON ra.exam_id = e.exam_id
            WHERE e.exam_date = %(exam_date)s AND e.slot = %(slot)s
        );
        """
        return sql, params

    # -----------------------------------------
    # DEFAULT PATH (uses SQL templates)
    # -----------------------------------------
    template = SQL_TEMPLATES.get(intent)
    if not template:
        raise ValueError("No template found for intent: " + intent)

    params: Dict[str, Any] = {}
    exam_name_filter = ""
    date_slot_filter = ""

    # standard params
    if "usn" in entities:
        params["usn"] = entities["usn"]
    if "fac_id" in entities:
        params["fac_id"] = entities["fac_id"]
    if "room_id" in entities:
        params["room_id"] = entities["room_id"]
    if "exam_name" in entities:
        params["exam_name"] = entities["exam_name"]
        exam_name_filter = " AND e.exam_name = %(exam_name)s "

    if "exam_id" in entities:
        params["exam_id"] = int(entities["exam_id"])
    # date normalization
    normalized = normalize_date(entities)
    if normalized:
        params["exam_date"] = normalized

    # slot normalization
    if "slot" in entities:
        params["slot"] = entities["slot"]
    elif "slot_simple" in entities:
        params["slot"] = entities["slot_simple"]

    # build date-slot filter
    if "exam_date" in params and "slot" in params:
        date_slot_filter = " AND e.exam_date = %(exam_date)s AND e.slot = %(slot)s "
    elif "exam_date" in params:
        date_slot_filter = " AND e.exam_date = %(exam_date)s "

    sql = template.format(
        exam_name_filter=exam_name_filter,
        date_slot_filter=date_slot_filter,
    )

    return sql, params


# -------------------------
# Main NL->SQL pipeline
# -------------------------

def nl_to_sql_and_run(nl_text: str, execute: bool = True) -> Dict[str, Any]:
    text = nl_text.strip()
    
    # --------------------------------------------------
    # 1. regex-based intent + entity extraction
    # --------------------------------------------------
    intent = classify_intent(text)
    entities = extract_entities(text)

    # --------------------------------------------------
    # 2. decide if LLM fallback is needed
    # --------------------------------------------------
    need_entity_fallback = False

    if intent.startswith("student"):
        if "usn" not in entities:
            need_entity_fallback = True

    if intent.startswith("faculty"):
        if "fac_id" not in entities and not (
            "exam_date" in entities or "slot" in entities or "exam_date_raw" in entities
        ):
            need_entity_fallback = True

    if intent in ("unallocated_students", "unused_rooms", "who_is_in_room", "room_remaining_capacity"):
        if intent == "unallocated_students" and "exam_id" not in entities:
            need_entity_fallback = True

        if intent == "unused_rooms" and not (
            "exam_date" in entities or "exam_date_raw" in entities or "slot" in entities or "slot_simple" in entities
        ):
            need_entity_fallback = True

        if intent in ("who_is_in_room", "room_remaining_capacity") and "room_id" not in entities:
            need_entity_fallback = True

    # --------------------------------------------------
    # 3. LLM fallback (optional)
    # --------------------------------------------------
    llm_json = None
    if need_entity_fallback:
        prompt = f"Extract JSON intent+entities (strict) from the user message exactly, no explanations:\n{text}"
        raw = call_local_llama(prompt)

        if raw:
            sanitized = sanitize_llama_output(raw)
            if sanitized:
                llm_json = sanitized
                for k, v in sanitized.get("entities", {}).items():
                    if k not in entities and v:
                        entities[k] = v

    # --------------------------------------------------
    # 4. clean invalid values
    # --------------------------------------------------
    for k in list(entities.keys()):
        if str(entities[k]).strip().lower() in INVALID_ENTITY_VALUES:
            del entities[k]

    # --------------------------------------------------
    # 5. heuristics fallback
    # --------------------------------------------------
    low = text.lower()

    if intent == "faculty_invigilation" and "slot" not in entities:
        if "afternoon" in low or "14:30" in low or "2:30" in low:
            entities["slot"] = "14:30-16:30"
        else:
            entities["slot"] = "09:30-11:30"

    # --------------------------------------------------
    # 6. UNUSED ROOMS special fix (slot inference)
    # --------------------------------------------------

    if intent == "faculty_invigilation":
        if "slot" not in entities and "slot_simple" not in entities:
            low = text.lower()
            if "afternoon" in low or "14:30" in low:
                entities["slot"] = "14:30-16:30"
            else:
                entities["slot"] = "09:30-11:30"

    if intent == "unused_rooms":
        # if user gave only date, slot will be None → prepare_query handles it
        pass

    # set exam_date if user said “today/tomorrow”
    low = text.lower()
    if "today" in low:
        entities["exam_date"] = datetime.now().strftime("%Y-%m-%d")
    elif "tomorrow" in low:
        entities["exam_date"] = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")




    # --------------------------------------------------
    # 7. normalize date
    # --------------------------------------------------
    if "exam_date_raw" in entities and "exam_date" not in entities:
        nd = normalize_date(entities)
        if nd:
            entities["exam_date"] = nd

    # --------------------------------------------------
    # 8. FINAL: Build SQL
    # --------------------------------------------------
    try:
        sql, params = prepare_query(intent, entities)
    except Exception as e:
        return {"error": f"Prepare query failed: {e}", "intent": intent, "entities": entities}

    # --------------------------------------------------
    # 9. EXECUTION
    # --------------------------------------------------
    if execute:
        try:
            rows = run_sql_fetchall(sql, params)
            return {"intent": intent, "entities": entities, "sql": sql, "params": params, "rows": rows}
        except Exception as e:
            return {"error": f"SQL execution error: {e}", "intent": intent, "entities": entities, "sql": sql, "params": params}
    else:
        return {"intent": intent, "entities": entities, "sql": sql, "params": params}


# -------------------------
# DB health check helper
# -------------------------

def check_db_health():
    required = ["admin", "exam", "faculty", "room_allocation", "room_details", "student", "student_allocation"]
    try:
        conn = db_connect()
        cur = conn.cursor()
        cur.execute("SHOW TABLES")
        tables = {r[0] for r in cur.fetchall()}
        cur.close()
        conn.close()
        missing = [t for t in required if t not in tables]
        return missing
    except Exception as e:
        return f"DB connection error: {e}"

# -------------------------
# Test harness
# -------------------------
if __name__ == "__main__":
    ok, sample = False, None
    try:
        resp = client.chat.completions.create(model="llama3.2", messages=[{"role":"user","content":"ping"}], temperature=0)
        sample = resp.choices[0].message.content
        ok = "pong" in sample.lower() or sample.strip().lower().startswith("*ping*") or "ping" in sample.lower()
    except Exception as e:
        ok = False
        sample = str(e)
    print("Local Llama connected:", ok, "sample:", sample)

    # DB health quick check
    print("DB configured:", MYSQL_CONFIG["database"]) 
    health = check_db_health()
    print("DB health missing tables or error:", health)

    examples = [
        "Where is my exam DBMS? USN 1RV24IS001",
        "Which bench am I sitting on for CLOUD? 1RV24IS135",
        "FAC002 where do i invigilate on 2025-10-26 at 14:30",
        "Which rooms are free on 2025-10-26 at 14:30?",
        "How many students are not allocated for exam_id 5?",
        "Iam 1RV23IS007, which exam i have on 2025-10-26 at 09:30-11:30 and where",
        "Where should I go tomorrow afternoon? I am FAC007",
        "which rooms are empty on 2025-10-26 at 14:30"
    ]

    for ex in examples:
        print('\n---')
        print('NL:', ex)
        out = nl_to_sql_and_run(ex, execute=True)
        print('Intent:', out.get('intent'))
        print('Entities:', out.get('entities'))
        print('SQL (preview):', (out.get('sql') or '').strip()[:400])
        print('Params:', out.get('params'))
        if out.get('rows') is not None:
            print('Rows (sample):', out.get('rows')[:10])
        else:
            print('Error:', out.get('error'))


Local Llama connected: True sample: *ping*

Response time: 200 ms
DB configured: ml_db_collection
DB health missing tables or error: []

---
NL: Where is my exam DBMS? USN 1RV24IS001
Intent: student_exam_room
Entities: {'usn': '1RV24IS001', 'exam_name': 'DBMS'}
SQL (preview): SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, e.exam_date, e.exam_name
FROM student_allocation sa
JOIN exam e ON sa.exam_id = e.exam_id
WHERE sa.usn = %(usn)s  AND e.exam_name = %(exam_name)s
Params: {'usn': '1RV24IS001', 'exam_name': 'DBMS'}
Rows (sample): [{'room_id': 'IS112B', 'bench_number': 17, 'bench_place': 'middle', 'time_slot': '09:30-11:30', 'exam_date': datetime.date(2025, 10, 25), 'exam_name': 'DBMS'}]

---
NL: Which bench am I sitting on for CLOUD? 1RV24IS135
Intent: student_exam_bench
Entities: {'usn': '1RV24IS135', 'exam_name': 'CLOUD'}
SQL (preview): SELECT sa.room_id, sa.bench_number, sa.bench_place, sa.time_slot, e.exam_date, e.exam_name
FROM student_allocation sa
JOIN exam e 